# 🚀 Phi-2 LoRA Fine-tuning for RunPod A40
### Multi-QA Dataset Training with Optimizations

**Features:**
- ✅ RunPod A40 최적화 설정
- ✅ 여러 QA 파일 자동 병합
- ✅ 메모리 효율적인 4bit 양자화
- ✅ A40 GPU에 맞춘 배치 크기
- ✅ WandB 통합 모니터링
- ✅ 자동 모델 저장 및 업로드

In [ ]:
# ================================================================
# 🔧 1. 환경 설정 및 라이브러리 설치 (RunPod A40 최적화)
# ================================================================

# 최신 PyTorch CUDA 12.1 설치 (A40 최적화)
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install transformers datasets peft accelerate bitsandbytes
!pip install wandb trl xformers flash-attn --no-build-isolation
!pip install --upgrade huggingface_hub

# 환경 변수 설정 (A40 최적화)
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
# A40에서 메모리 효율성을 위한 설정
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:512'

print("✅ 라이브러리 설치 완료!")

In [ ]:
# ================================================================
# 📊 2. GPU 환경 확인 및 최적화 설정
# ================================================================

import random
import torch
import json
import pandas as pd
import gc
from datasets import Dataset, concatenate_datasets
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, TaskType, prepare_model_for_kbit_training
import warnings
warnings.filterwarnings("ignore")

# GPU 정보 상세 확인
print(f"🚀 CUDA 사용 가능: {torch.cuda.is_available()}")
print(f"🔢 GPU 개수: {torch.cuda.device_count()}")
if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        props = torch.cuda.get_device_properties(i)
        print(f"📱 GPU {i}: {props.name}")
        print(f"💾 메모리: {props.total_memory / 1024**3:.1f} GB")
        print(f"🔧 Compute Capability: {props.major}.{props.minor}")

# PyTorch 버전 확인
print(f"⚡ PyTorch 버전: {torch.__version__}")
print(f"🎯 CUDA 버전: {torch.version.cuda}")

# A40에 최적화된 설정
DEVICE_NAME = torch.cuda.get_device_name() if torch.cuda.is_available() else "CPU"
IS_A40 = "A40" in DEVICE_NAME
IS_V100 = "V100" in DEVICE_NAME
IS_A100 = "A100" in DEVICE_NAME

print(f"🎮 감지된 GPU: {DEVICE_NAME}")
print(f"🔍 A40 최적화 모드: {IS_A40}")

In [ ]:
# ================================================================
# 🔐 3. Hugging Face 로그인 및 WandB 설정
# ================================================================

from huggingface_hub import login
import wandb
import getpass

# Hugging Face 토큰 입력
print("🔑 Hugging Face 토큰을 입력하세요:")
print("토큰 생성: https://huggingface.co/settings/tokens")
hf_token = getpass.getpass("HF 토큰: ")
login(token=hf_token)
print("✅ Hugging Face 로그인 완료!")

# WandB 설정 (선택사항)
use_wandb = input("WandB 사용하시겠습니까? (y/n): ").lower() == 'y'
if use_wandb:
    wandb_token = getpass.getpass("WandB API 키: ")
    wandb.login(key=wandb_token)
    print("✅ WandB 로그인 완료!")
else:
    print("⏭️ WandB 스킵")

In [ ]:
# ================================================================
# 🤖 4. 모델 및 토크나이저 로드 (A40 최적화)
# ================================================================

model_name = "microsoft/phi-2"
print(f"🔄 모델 로딩 중: {model_name}")

# 토크나이저 설정
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# A40 최적화 4bit 양자화 설정 (48GB VRAM 활용)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# A40에서 Flash Attention 사용 (성능 향상)
try:
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True,
        torch_dtype=torch.bfloat16,
        attn_implementation="flash_attention_2",
    )
    print("⚡ Flash Attention 2 활성화됨")
except:
    print("⚠️ Flash Attention 2 실패, 기본 attention 사용")
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True,
        torch_dtype=torch.bfloat16
    )

# kbit 훈련용 준비
model = prepare_model_for_kbit_training(model)

# 메모리 정리
torch.cuda.empty_cache()
gc.collect()

print("✅ 모델 로딩 완료!")
print(f"💾 현재 VRAM 사용량: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")

In [ ]:
# ================================================================
# ⚙️ 5. LoRA 설정 (A40 최적화)
# ================================================================

# A40 48GB 메모리를 활용한 더 큰 LoRA 설정
lora_config = LoraConfig(
    r=32 if IS_A40 else 16,
    lora_alpha=64 if IS_A40 else 32,
    target_modules=[
        "q_proj",
        "k_proj", 
        "v_proj",
        "dense",
        "fc1",
        "fc2"
    ],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

# LoRA 어댑터 추가
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

print(f"⚡ LoRA rank: {lora_config.r}")
print(f"🎯 Target modules: {len(lora_config.target_modules)}개")
print(f"💾 LoRA 후 VRAM: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")

print("\n✅ Part 1 완료! 이제 test_part2.ipynb를 실행하세요.")

In [ ]:
# ================================================================
# 📁 6. 여러 QA 데이터셋 로드 및 병합
# ================================================================

qa_data = json.load(open('qa_pairs.json', 'r', encoding='utf-8'))
# 데이터 셔플
random.seed(42)
random.shuffle(qa_data)
print(f"🔀 데이터 셔플 완료")

def format_qa_pair(example):
    """QA 쌍을 훈련용 텍스트로 포맷팅"""
    question = example['question']
    answer = example['answer']
    context = example['context']
    # Context를 포함한 Phi-2에 적합한 프롬프트 템플릿
    formatted_text = f"Context: {context}\n\nQuestion: {question}\nAnswer: {answer}<|endoftext|>"

    return {"text": formatted_text}

# 데이터셋 생성
dataset = Dataset.from_list(qa_data)
dataset = dataset.map(format_qa_pair)

# 샘플 데이터 확인
print(f"\n📝 샘플 데이터:")
print(dataset[999]["text"])

In [ ]:
# ================================================================
# 🎯 8. 토크나이징 (A40 최적화)
# ================================================================

def tokenize_function_optimized(examples):
    """A40 최적화 토크나이징 함수"""
    texts = examples["text"] if isinstance(examples["text"], list) else [examples["text"]]
    
    # A40 48GB를 활용해 더 긴 시퀀스 지원
    max_length = 768 if IS_A40 else 512
    
    tokenized = tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=max_length,
        return_tensors=None
    )
    
    # labels = input_ids (Causal LM)
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

print("🔄 토크나이징 시작...")
print(f"📏 최대 시퀀스 길이: {768 if IS_A40 else 512}")

# 병렬 토크나이징
tokenized_dataset = dataset.map(
    tokenize_function_optimized,
    batched=True,
    num_proc=8 if IS_A40 else 4,
    remove_columns=[col for col in dataset.column_names if col != "source"]
)

print("✅ 토크나이징 완료!")
print(f"📊 토큰화된 샘플 수: {len(tokenized_dataset)}")

# 토큰 길이 통계
token_lengths = [len(item['input_ids']) for item in tokenized_dataset[:min(1000, len(tokenized_dataset))]]
print(f"📏 평균 토큰 길이: {sum(token_lengths)/len(token_lengths):.1f}")
print(f"📏 최대 토큰 길이: {max(token_lengths)}")

In [ ]:
# ================================================================
# ⚙️ 9. 훈련 설정 (A40 최적화)
# ================================================================

from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

# 데이터 콜레이터
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# WandB 설정 확인
try:
    use_wandb = 'use_wandb' in globals() and use_wandb
except:
    use_wandb = False

# 훈련 인자
training_args = TrainingArguments(
    output_dir="./phi2-multi-qa-lora",
    num_train_epochs=3,
    
    # 배치 크기 (안정성과 성능의 균형)
    per_device_train_batch_size=6 if IS_A40 else 4,  # A40에서는 6도 가능
    gradient_accumulation_steps=4,  # 총 effective batch = 24 or 16
    
    # 학습률 (LoRA에 최적화)
    learning_rate=8e-5,  # 중간값으로 조정
    weight_decay=0.01,
    warmup_ratio=0.03,  # warmup_steps 제거하고 비율만 사용
    
    # 로깅 및 저장
    logging_steps=25,  # 너무 자주 로깅하면 성능 저하
    save_steps=200 if IS_A40 else 500,
    save_total_limit=3,
    
    # 평가 설정 추가 (중요!)
    eval_strategy="steps",
    eval_steps=200,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    
    # 최적화 설정
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    max_grad_norm=1.0,  # gradient clipping 추가
    
    # 메모리 최적화
    dataloader_pin_memory=False,  # 4bit 양자화시 False
    remove_unused_columns=False,
    bf16=True,
    gradient_checkpointing=True,
    
    # 기타 설정
    report_to="wandb" if use_wandb else None,
    run_name=f"phi2-multi-qa-{len(all_qa_data)}samples",
    push_to_hub=False,
)

print(f"💾 체크포인트 저장 간격: {training_args.save_steps} 스텝")
print(f"📊 로깅 간격: {training_args.logging_steps} 스텝")

In [ ]:
# ================================================================
# 🚀 10. 트레이너 설정 및 훈련 시작
# ================================================================
import gc

# TRL SFTTrainer 사용 (더 안정적)
try:
    from trl import SFTTrainer
    
    trainer = SFTTrainer(
        model=model,
        train_dataset=dataset,
        args=training_args,
        tokenizer=tokenizer,
        packing=False,
        dataset_text_field="text"
    )
    print("✅ SFTTrainer 설정 완료")
    
except Exception as e:
    print(f"⚠️ SFTTrainer 실패: {e}")
    print("🔄 기본 Trainer로 전환")
    
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator
    )
    print("✅ 기본 Trainer 설정 완료")

# 훈련 전 메모리 상태
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    gc.collect()
    print(f"💾 훈련 전 VRAM: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
    total_memory = torch.cuda.get_device_properties(0).total_memory
    free_memory = total_memory - torch.cuda.memory_allocated()
    print(f"💾 VRAM 여유: {free_memory / 1024**3:.2f} GB")

# 훈련 시작!
print("\n🚀 훈련 시작!")
print(f"🎯 총 샘플 수: {len(all_qa_data)}")
print(f"📊 에포크: {training_args.num_train_epochs}")
estimated_time = len(all_qa_data) * training_args.num_train_epochs / (batch_size * grad_acc_steps) * 2 / 60
print(f"⏱️ 예상 소요 시간: {estimated_time:.1f}분")

# 훈련 실행
trainer.train()

print("🎉 훈련 완료!")

In [ ]:
# ================================================================
# 💾 11. 모델 저장 및 업로드
# ================================================================

# LoRA 어댑터 저장
output_dir = "./phi2-multi-qa-lora-final"
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"✅ 모델 저장 완료: {output_dir}")

# 모델 크기 확인
import os
total_size = 0
for dirpath, dirnames, filenames in os.walk(output_dir):
    for filename in filenames:
        filepath = os.path.join(dirpath, filename)
        total_size += os.path.getsize(filepath)

print(f"📁 저장된 모델 크기: {total_size / 1024**2:.1f} MB")

# 훈련 완료 후 메모리 정리
torch.cuda.empty_cache()
gc.collect()
print(f"💾 최종 VRAM 사용량: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")

print("\n🎊 모든 작업이 완료되었습니다!")
print(f"📂 저장 위치: {os.path.abspath(output_dir)}")

In [ ]:
# ================================================================
# 10. 선택사항: 허깅페이스 허브에 업로드
# ================================================================

# 모델 업로드
model.push_to_hub("cometlee39/phi2-lora-finetuned")
tokenizer.push_to_hub("cometlee39/phi2-lora-finetuned")

print("\n모든 과정이 완료되었습니다!")
print("LoRA 어댑터가 './phi2-lora-adapter' 폴더에 저장되었습니다.")